# 📓 The GenAI Revolution Cookbook

**Title:** Data Science Jobs: How to Get Hired with No Experience (2025)

**Description:** Land your first data science role faster with a recruiter-backed 90-day roadmap, portfolio projects, ATS-proof resume toolkit, and interview scripts.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



When you ask a language model to return structured data, you often get back prose, malformed JSON, or outputs that break your parser. This happens because models prioritize natural language fluency over strict format adherence, especially under long context or high temperature. This guide shows you how to prompt for schema-compliant JSON using system-role constraints, delimiters, and minimal examples, so your downstream services can parse outputs reliably every time.

## What Problem Are We Solving?

You need your model to return a JSON object with specific fields. `title` and `severity`, for example. But instead of clean JSON, you get:

- Prose wrappers: "Here's the summary: {'title': '...', 'severity': '...'}"
- Missing fields or extra keys the schema doesn't define
- Format drift under long inputs, where the model reverts to natural language mid-response

This breaks parsers, crashes pipelines, and forces you to write brittle regex fallbacks. The root cause is that instruction priority dilutes as context grows, and sampling randomness lets the model "choose" between formats.

## What's Actually Happening Under the Hood

Language models process your prompt as a sequence of tokens and predict the next token based on learned probabilities. When you ask for JSON, the model balances that instruction against:

- **Instruction dilution.** Long user messages or multi-turn context push the format constraint further back in the attention window, reducing its influence on generation.
- **Sampling temperature.** Higher temperature increases randomness, making the model more likely to deviate from strict structure in favor of fluent prose.
- **Lack of examples.** Without a concrete demonstration, the model infers what "JSON" means from training data, which includes many malformed or prose-wrapped examples.
- **Ambiguous boundaries.** If input text and instructions blend together, the model may treat part of your data as additional instructions or vice versa.

The result is that format compliance competes with other learned behaviors, and without strong constraints, the model drifts toward natural language.

```mermaid
flowchart TD
  A[System: Output valid JSON only] --> B[User: Task + long context]
  B --> C[Model infers priorities]
  C -->|High temp / no examples| D[Format drift]
  C -->|Schema reminder + examples| E[Compliant JSON]
```

## How to Fix It: Prompt Patterns and Examples

### Before: Weak Format Constraint

This prompt mixes instruction and input without clear boundaries, and provides no schema or example.

**Prompt:**

In [ ]:
Summarize this support ticket as JSON with title and severity fields.

Ticket: Customer reports checkout fails when applying promo code SAVE20. Error message: "Invalid discount." Affects 15% of users since yesterday's deploy.

**Output:**

In [ ]:
Here's a summary of the ticket:

Title: Checkout fails on promo code
Severity: This is a high-priority issue affecting 15% of users.

The model returned prose instead of JSON because the instruction was vague and no example anchored the expected format.

### After: Strict Schema Enforcement

This prompt isolates the system constraint, restates the schema immediately before the input, wraps the input in delimiters, and forbids extra prose.

**Prompt:**

In [ ]:
System: You are a JSON generator. Return ONLY valid minified JSON. No prose, no explanations.

User: Use this exact schema: {"title": "string", "severity": "low" | "med" | "high"}

Input:

Customer reports checkout fails when applying promo code SAVE20. Error message: "Invalid discount." Affects 15% of users since yesterday's deploy.

**Output:**

```json
{"title":"Checkout fails on promo code SAVE20","severity":"high"}
```

The model now returns clean, parseable JSON because the constraints are explicit, the schema is restated right before the input, and delimiters prevent instruction bleed.

### Best Practices for Structured Output Prompts

1. **Isolate system-level constraints.** Use a system message or prefix to declare output-only behavior. Restate the schema immediately before the input to maximize recency bias.
2. **Wrap inputs with delimiters.** Triple backticks, XML tags, or clear separators prevent the model from treating data as instructions.
3. **Keep temperature low.** Set temperature to 0 or 0.1 to minimize sampling randomness. Add one positive example and one negative example if the task is ambiguous.
4. **Forbid extra prose.** Explicitly state "no explanations" or "output only JSON" to suppress the model's tendency to add commentary.

### When to Use This Pattern

- Downstream parsers expect strict JSON and cannot tolerate prose wrappers or missing fields.
- Long context or multi-turn conversations cause format drift.
- You need deterministic, reproducible outputs for production pipelines.
- The model has no prior examples of your specific schema and must infer structure from the prompt alone.

## Key Takeaways

Reliable structured outputs require explicit schema constraints, clear input boundaries, and low-temperature sampling. By isolating system instructions, restating the schema close to the input, and wrapping data in delimiters, you eliminate ambiguity and force the model to prioritize format compliance over fluency. This pattern is essential whenever your application depends on parseable, schema-compliant JSON to function correctly.